## Imports and Data Fetching

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 1
%aimport nanslice
%aimport nanslice.jupyter
%aimport QUIT.interfaces.rufis
import nibabel as nib
import numpy as np
import requests
import gzip
from os.path import isfile

## Setup

In [ ]:
if not isfile('classes.mnc'):
    print('Downloading classes')
    params = {'download_for_real':'[Start Download!]',
              'do_download_alias':'phantom_1.0mm_normal_crisp',
              'format_value':'minc',
              'who_name': 'Tobias Wood',
              'who_institution': 'KCL',
              'who_email': 'tobias.wood@kcl.ac.uk'}
    response = requests.get(url='http://brainweb.bic.mni.mcgill.ca/cgi/brainweb1', params=params)
    minc_file = open('classes.mnc', 'wb')
    minc_file.write(response.content)
if not isfile('rf20_C.mnc'):
    print('Downloading B1')
    params = {'download_for_real':'[Start Download!]',
              'do_download_alias':'rf20_C.mnc.gz',
              'format_value':'minc',
              'zip_value':'none',
              'who_name': 'Tobias Wood',
              'who_institution': 'KCL',
              'who_email': 'tobias.wood@kcl.ac.uk'}
    response = requests.get(url='https://brainweb.bic.mni.mcgill.ca/cgi/brainweb1', params=params)
    b1_file = open('rf20_C.mnc', 'wb')
    b1_file.write(response.content)
classes = nanslice.Layer('classes.mnc')
b1_minc = nanslice.Layer('rf20_C.mnc')

In [ ]:
# 0=Background
# 1=CSF
# 2=Grey Matter
# 3=White Matter
# 4=Fat
# 5=Muscle/Skin
# 6=Skin
# 7=Skull
# 8=Glial Matter
# 9=Connective
def classes_to_params(M0Vals, T1Vals, T2vals, subsamp=1):
    class_data = classes.image.get_data().astype('int32')
    M0data = np.choose(class_data[::subsamp,::subsamp,::subsamp], M0vals).astype('float32')
    T1data = np.choose(class_data[::subsamp,::subsamp,::subsamp], T1vals).astype('float32')
    T2data = np.choose(class_data[::subsamp,::subsamp,::subsamp], T2vals).astype('float32')
    B1data = b1_minc.image.get_data().astype('float32')[::subsamp,::subsamp,::subsamp]
    # PDdata = np.array(list(map(PDFunc, classes.image.get_data())))
    M0image = nib.nifti1.Nifti1Image(M0data, affine=classes.image.affine)
    T1image = nib.nifti1.Nifti1Image(T1data, affine=classes.image.affine)
    T2image = nib.nifti1.Nifti1Image(T2data, affine=classes.image.affine)
    B1image = nib.nifti1.Nifti1Image(B1data, affine=classes.image.affine)
    nib.save(M0image, 'M0.nii.gz')
    nib.save(T1image, 'T1.nii.gz')
    nib.save(T2image, 'T2.nii.gz')
    nib.save(B1image, 'B1.nii.gz')

In [ ]:
prep_pulses = {
    'inv': {'FAeff': 175.3, 'T_long': 0.0365, 'T_trans': 0.004, 'int_b1_sq': 53390.0},
    't2-20': {'FAeff': 0.07, 'T_long': 0.0055, 'T_trans': 0.020, 'int_b1_sq': 279600.0},
    't2-40': {'FAeff': 0.07, 'T_long': 0.0055, 'T_trans': 0.040, 'int_b1_sq': 279600.0},
    't2-60': {'FAeff': 0.07, 'T_long': 0.0055, 'T_trans': 0.060, 'int_b1_sq': 279600.0},
    't2-80': {'FAeff': 0.07, 'T_long': 0.0055, 'T_trans': 0.080, 'int_b1_sq': 279600.0},
    't2-120': {'FAeff': 0.07, 'T_long': 0.0055, 'T_trans': 0.120, 'int_b1_sq': 279600.0},
    't2-160': {'FAeff': 0.07, 'T_long': 0.0055, 'T_trans': 0.160, 'int_b1_sq': 279600.0},
    't2-inv': {'FAeff': 180., 'T_long': 0.0055, 'T_trans': 0.020, 'int_b1_sq': 279600.0},
    't1-20': {'FAeff': 20, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-45': {'FAeff': 45, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-60': {'FAeff': 60, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-75': {'FAeff': 75, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-90': {'FAeff': 90, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-120': {'FAeff': 120, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-180': {'FAeff': 180, 'T_long': 0.036487, 'T_trans': 0.004396, 'int_b1_sq': 0},
    't1-360': {'FAeff': 360, 'T_long': 0.03, 'T_trans': 0.004, 'int_b1_sq': 0},
    'null': {'FAeff': 0.0, 'T_long': 0.0, 'T_trans': 0.0, 'int_b1_sq': 0.0},
    'delay-400': {'FAeff': 0.0, 'T_long': 0.4, 'T_trans': 0.0, 'int_b1_sq': 0.0},
    'delay-800': {'FAeff': 0.0, 'T_long': 0.8, 'T_trans': 0.0, 'int_b1_sq': 0.0}
}

## Simulate and Display Image

In [ ]:
M0vals = np.array([0, 1, 0.8, 0.7, 0, 0, 0, 0, 0, 0])
T1vals = np.array([0, 3.0, 1.3, 0.9, 0, 0, 0, 0, 0, 0])
T2vals = np.array([0, 0.5, 0.08, 0.06, 0, 0, 0, 0, 0, 0])
classes_to_params(M0vals, T1vals, T2vals, 2)

In [ ]:
seq = {
    "RUFIS": {
        "TR": 0.002012,
        "Tramp": 0.01,
        "Tspoil": 0.05,
        "Trf": [28e-6, 28e-6, 28e-6],
        "spokes_per_seg": 48,
        "groups_per_seg": [1, 1, 1],
        "FA": [2, 8, 8],
        "prep": ["null", "null", "t1-360"],
        "prep_pulses": prep_pulses
    }
}
region='45,0,0,1,109,91'
sim_result = QUIT.interfaces.rufis.SteadyStateSim(sequence=seq, in_file='sim.nii.gz', noise=0.0005, M0='M0.nii.gz', T1='T1.nii.gz', B1='B1.nii.gz', mask='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.timeseries(sim_result.outputs.out_file, clim=[0,0.025]))
mupa = QUIT.interfaces.rufis.SteadyState(sequence=seq, in_file='sim.nii.gz', mask_file='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.compare('M0.nii.gz', 'RUFIS_SS_M0.nii.gz', axis='z', clim=[0, 1.25], title='M0'))
display(nanslice.jupyter.compare('T1.nii.gz', 'RUFIS_SS_T1.nii.gz', axis='z', clim=[0.5, 1.5], title='T1'))
display(nanslice.jupyter.compare('B1.nii.gz', 'RUFIS_SS_B1.nii.gz', axis='z', clim=[0.5, 1.5], title='B1'))

## Emil's November Data

In [ ]:
seq = {
    'MUPA': {
        'TR': 2.34e-3,
        'Tramp': 10e-3,
        'spokes_per_seg': 256,
        'groups_per_seg': [1, 1, 1, 1, 1, 1, 1],
        'FA': [2, 2, 2, 2, 2, 2, 2],
        'Trf': [24, 24, 24, 24, 24, 24, 24],
        'prep': ['inv', 'null', 'null', 'null', 't2-40', 't2-80', 't2-160'],
        'prep_pulses': prep_pulses}
}
region='45,0,0,1,109,91'
sim_result = QUIT.interfaces.rufis.MUPASim(sequence=seq, in_file='sim.nii.gz', noise=0.0005, M0='M0.nii.gz', T1='T1.nii.gz', T2='T2.nii.gz', B1='B1.nii.gz', mask='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.timeseries(sim_result.outputs.out_file, clim=[-0.01,0.02]))
mupa = QUIT.interfaces.rufis.MUPA(sequence=seq, in_file='sim.nii.gz', mask_file='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.compare('M0.nii.gz', 'MUPA_M0.nii.gz', axis='z', clim=[0, 1.25], title='M0'))
display(nanslice.jupyter.compare('T1.nii.gz', 'MUPA_T1.nii.gz', axis='z', clim=[0.5, 1.5], title='T1'))
display(nanslice.jupyter.compare('T2.nii.gz', 'MUPA_T2.nii.gz', axis='z', clim=[0.0, 0.15], title='T2'))

## 4 Point

In [ ]:
seq = {
    'MUPA': {
        'TR': 2.34e-3,
        'Tramp': 10e-3,
        'spokes_per_seg': 512,
        'groups_per_seg': [1, 1, 1, 1],
        'FA': [2, 2, 2, 2],
        'Trf': [24, 24, 24, 24],
        'prep': ['inv', 'delay-800', 't2-80', 'delay-800'],
        'prep_pulses': prep_pulses}
}
region='45,0,0,1,109,91'
sim_result = QUIT.interfaces.rufis.MUPASim(sequence=seq, in_file='sim.nii.gz', noise=0.0005, M0='M0.nii.gz', T1='T1.nii.gz', T2='T2.nii.gz', B1='B1.nii.gz', mask='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.timeseries(sim_result.outputs.out_file, clim=[-0.01,0.025]))
mupa = QUIT.interfaces.rufis.MUPA(sequence=seq, in_file='sim.nii.gz', mask_file='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.compare('M0.nii.gz', 'MUPA_M0.nii.gz', axis='z', clim=[0, 1.25], title='M0'))
display(nanslice.jupyter.compare('T1.nii.gz', 'MUPA_T1.nii.gz', axis='z', clim=[0.5, 1.5], title='T1'))
display(nanslice.jupyter.compare('T2.nii.gz', 'MUPA_T2.nii.gz', axis='z', clim=[0.0, 0.15], title='T2'))

## 4 Point with Groups

In [ ]:
seq = {
    'MUPA': {
        'TR': 2.34e-3,
        'Tramp': 10e-3,
        'spokes_per_seg': 512,
        'groups_per_seg': [1, 1, 4, 1],
        'FA': [2, 2, 2, 2],
        'Trf': [24, 24, 24, 24],
        'prep': ['inv', 'delay-800','t2-80', 'delay-800'],
        'prep_pulses': prep_pulses}
}
region='45,0,0,1,109,91'
sim_result = QUIT.interfaces.rufis.MUPASim(sequence=seq, in_file='sim.nii.gz', noise=0.0005, M0='M0.nii.gz', T1='T1.nii.gz', T2='T2.nii.gz', B1='B1.nii.gz', mask='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.timeseries(sim_result.outputs.out_file, clim=[-0.01,0.025]))
mupa = QUIT.interfaces.rufis.MUPA(sequence=seq, in_file='sim.nii.gz', mask_file='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.compare('M0.nii.gz', 'MUPA_M0.nii.gz', axis='z', clim=[-0.01, 1.25], title='M0'))
display(nanslice.jupyter.compare('T1.nii.gz', 'MUPA_T1.nii.gz', axis='z', clim=[0.5, 1.5], title='T1'))
display(nanslice.jupyter.compare('T2.nii.gz', 'MUPA_T2.nii.gz', axis='z', clim=[0.0, 0.15], title='T2'))

## T2-Prep Only

In [ ]:
seq = {
    'MUPA': {
        'TR': 2.34e-3,
        'Tramp': 10e-3,
        'spokes_per_seg': 512,
        'groups_per_seg': [4, 4, 1, 1],
        'FA': [2, 2, 2, 2],
        'Trf': [24, 24, 24, 24],
        'prep': ['t2-40', 't2-80', 'null', 'delay-800'],
        'prep_pulses': prep_pulses}
}
region='45,0,0,1,109,91'
sim_result = QUIT.interfaces.rufis.MUPASim(sequence=seq, in_file='sim.nii.gz', noise=0.0005, M0='M0.nii.gz', T1='T1.nii.gz', T2='T2.nii.gz', B1='B1.nii.gz', mask='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.timeseries(sim_result.outputs.out_file, clim=[0,0.025]))
mupa = QUIT.interfaces.rufis.MUPA(sequence=seq, in_file='sim.nii.gz', mask_file='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.compare('M0.nii.gz', 'MUPA_M0.nii.gz', axis='z', clim=[-0.01, 1.25], title='M0'))
display(nanslice.jupyter.compare('T1.nii.gz', 'MUPA_T1.nii.gz', axis='z', clim=[0.5, 1.5], title='T1'))
display(nanslice.jupyter.compare('T2.nii.gz', 'MUPA_T2.nii.gz', axis='z', clim=[0.0, 0.15], title='T2'))

In [ ]:
FAe = np.degrees(np.arccos(np.exp(-0.00234/1)))
print(FAe)

In [ ]:
seq = {
    'MUPA': {
        'TR': 2.34e-3,
        'Tramp': 10e-3,
        'spokes_per_seg': 512,
        'groups_per_seg': [1, 1, 4],
        'FA': [3, 3, 3],
        'Trf': [24, 24, 24],
        'prep': ['inv', 'null','t2-60'],
        'prep_pulses': prep_pulses}
}
region='45,0,0,1,109,91'
sim_result = QUIT.interfaces.rufis.MUPASim(sequence=seq, in_file='sim.nii.gz', noise=0.0005, M0='M0.nii.gz', T1='T1.nii.gz', T2='T2.nii.gz', B1='B1.nii.gz', mask='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.timeseries(sim_result.outputs.out_file, clim=[-0.01,0.025]))
mupa = QUIT.interfaces.rufis.MUPA(sequence=seq, in_file='sim.nii.gz', mask_file='M0.nii.gz', subregion=region).run()
display(nanslice.jupyter.compare('M0.nii.gz', 'MUPA_M0.nii.gz', axis='z', clim=[-0.01, 1.25], title='M0'))
display(nanslice.jupyter.compare('T1.nii.gz', 'MUPA_T1.nii.gz', axis='z', clim=[0.5, 1.5], title='T1'))
display(nanslice.jupyter.compare('T2.nii.gz', 'MUPA_T2.nii.gz', axis='z', clim=[0.0, 0.15], title='T2'))